# IS602 -- Assignment 11 -- James Hamski

1. Take your solution from Homework 11 and complete the Monte Carlo step (step 6) in parallel.  There are many ways you can go about doing this, and I'm not looking for anything too complicated.  If you can get multiple processes crunching the data together, that is great.  Using IPython’s built-in tools would be a great method
2. Compare the timing for your solution in homework 11 and this parallel solution.  This is similar to what you did in homeworks 6 and 7.  Ideally, you'll see some speed improvement.  The amount you see will largely be based the capabilities of your hardware, and less on the software implementation.  There is additional overhead for running an operation in parallel, so speed gains will be more obvious with a larger number of calculations.

Setup note: You must run the following command at the terminal: 

$ ipcluster start -n 4

in addition to $ jupyter notebook

*Python Modules*

In [1]:
%pylab inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import ipyparallel

Populating the interactive namespace from numpy and matplotlib


*Data Import and Formatting* 

In [2]:
apple = pd.read_csv('apple.2011.csv', header=0, na_values='XXXXX', parse_dates=True)
apple.columns = ['Date', 'Last', 'PercentChange']

In [3]:
apple['Date'] = pd.to_datetime(apple['Date'])

*Generate random numbers with the same probability distribution as the Percent Change column*

In [4]:
apple_mean_df = apple.mean()
apple_std_df = apple.std()

In [5]:
apple_mean = apple_mean_df['PercentChange']
print apple_mean

0.000957355207171


In [6]:
apple_std = apple_std_df['PercentChange']
print apple_std

0.0165205562984


In [7]:
def GeneratePercentChange(mean, std):
    return np.random.normal(loc=mean, scale=std, size=20)

In [8]:
def PriceQuote(start, PercentChange):
    price = start
    for i in PercentChange:
        price = price+(price*i)
    return price

Value at Risk with 99% confidence:

In [9]:
def MonteCarloSimulation():
    prices = []
    for i in range(0, iterations):
        random_walk = GeneratePercentChange(apple_mean, apple_std)
        end_price = PriceQuote(apple_start_price, random_walk)
        prices.append(end_price)
        
    prices_array = np.array(prices)
    
    VaR_99 = np.percentile(prices_array, 1)
    
    return VaR_99

In [10]:
iterations = 100000
apple_start_price = int(apple['Last'].tail(1))
print MonteCarloSimulation()

346.558281725


In [11]:
%timeit MonteCarloSimulation()

1 loops, best of 3: 818 ms per loop


## Parallel Computing Task

First, confirm ipyparallel is working. 

In [12]:
from ipyparallel import Client
c = Client()
c.ids

set([0,1,2,3])

c[:].apply_sync(lambda: "Hello World")

['Hello World', 'Hello World', 'Hello World', 'Hello World']

In [13]:
iterations = 100000/4

In [14]:
c[:].apply_sync(MonteCarloSimulation())

TypeError: f must be callable, not <type 'numpy.float64'>